LINK del Repositorio: https://github.com/WilberBarcayaM/SIS421-02-23

In [17]:
!pip install transformers

In [18]:
!pip install datasets

In [19]:
from datasets import load_dataset #Esta función se utiliza para cargar conjuntos de datos
from transformers import BartTokenizer, BartForConditionalGeneration # Estas clases son parte de la biblioteca Transformers de Hugging Face y se utilizan para trabajar con el modelo BART
import torch
from torch.utils.data import Dataset, DataLoader #Estas clases son esenciales para manejar conjuntos de datos y crear dataloaders para el entrenamiento de modelos
from tqdm import tqdm #se utiliza para mostrar barras de progreso durante el entrenamiento


In [24]:
# Cargar el conjunto de datos
dataset = load_dataset("Areeb123/drug_reviews")

# Limitar el porcentaje de datos después de cargar el conjunto de datos
percentage = 20  # ajusta según sea necesario
dataset['train'] = dataset['train'].shuffle(seed=42).select([i for i in range(int(len(dataset['train'])*percentage/100))])
dataset['test'] = dataset['test'].shuffle(seed=42).select([i for i in range(int(len(dataset['test'])*percentage/100))])
dataset['validation'] = dataset['validation'].shuffle(seed=42).select([i for i in range(int(len(dataset['validation'])*percentage/100))])

# Seleccionar las columnas relevantes
selected_columns = ['drugName', 'review']
dataset = dataset.select_columns(selected_columns)

# Acceder a las primeras 5 filas del conjunto de datos
primeras_filas = dataset['train'][:5]
print("Primeras 5 filas:")
print(primeras_filas)


# Crear instancia del tokenizer y del modelo BART
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base") #se encarga de dividir el texto en unidades más pequeñas llamadas "tokens" y asignarles identificadores numéricos únicos
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base") #Se crea una instancia del modelo BART para generación condicional. Este modelo está preentrenado en tareas de generación de texto y puede ser afinado para tareas específicas,



Primeras 5 filas:
{'drugName': ['Lamotrigine', 'Lysine', 'Varenicline', 'Bisacodyl', 'Donepezil'], 'review': ['"I started on Lamictal after having manic episodes on Zoloft as well as Wellbutrin. I was a heavy cigarette and marijuana smoker. In the first couple weeks of starting Lamictal, I was able to quit smoking weed and cigarettes!! I didn\'t have insomnia as I previously did when trying to quit marijuana. My mood was much improve. I had been suicidal before and I started exercising again on Lamictal. I developed an unknown rash about 2 years into Lamictal and stopped immediately. The rash disappeared. It looked like 8 mosquito bites all over my legs. I was forced to try Abilify which made me gain over 40 pounds and take naps during the day. Today, I have started back on Lamictal and am back on track with exercising and enjoying life."', '"There is no cure for HSV2 but you can suppress it with Lysine. You have to take it everyday and it just becomes part of your daily routine, like 

In [21]:
# Definir tu conjunto de datos como antes

class DrugRecommendationDataset(Dataset): #servirá para manejar y cargar los datos
    def __init__(self, dataset, tokenizer, max_length): #Este es el método constructor de la clase. Se llama cuando se crea una nueva instancia de la clase
        self.dataset = dataset #Almacena el conjunto de datos
        self.tokenizer = tokenizer #Almacena la instancia del tokenizador
        self.max_length = max_length #Almacena la longitud máxima para el padding

    def __len__(self):
        return len(self.dataset) #devuelve la longitud total del conjunto de datos

    def __getitem__(self, index): #Este método se utiliza para obtener un elemento del conjunto de datos en una posición específica index
        #Se extraen los textos de síntomas y el nombre del medicamento para el índice dado
        symptoms_text, drug_name = self.dataset['review'][index], self.dataset['drugName'][index]

        input_text = f"Recomendar medicamento para: {symptoms_text}"
        target_text = drug_name #El texto objetivo es simplemente el nombre del medicamento
        inputs = self.tokenizer( #Se utiliza el tokenizador para convertir el texto de entrada en una representación numérica que el modelo pueda entender
            input_text,
            max_length=self.max_length,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
        targets = self.tokenizer( #se tokeniza el texto objetivo nombre del medicamento
            target_text, #Es el texto que se desea tokenizar y convertir en una representación numérica.
            max_length=self.max_length, #Especifica la longitud máxima de la secuencia de salida después del tokenización
            return_tensors="pt", #Indica que se quiere que el resultado sea un tensor de PyTorch
            padding="max_length", #Especifica que se desea realizar el padding hasta la longitud máxima especificada
            truncation=True, #Indica que se debe truncar la secuencia si excede la longitud máxima especificada. Esto es útil para asegurar que todas las secuencias tengan la misma longitud.
        )
        return { #Se devuelve un diccionario con tres claves "nput_ids, attention_mask, y labels
            "input_ids": inputs["input_ids"].squeeze(),#Contiene los identificadores numéricos de los tokens de la secuencia de entrada, después de ser procesados por el tokenizador. En este caso, se usa .squeeze() para eliminar cualquier dimensión adicional y obtener un tensor unidimensional
            "attention_mask": inputs["attention_mask"].squeeze(), #Es una máscara de atención que indica qué elementos de la secuencia de entrada son tokens reales y cuáles son tokens de padding
            "labels": targets["input_ids"].squeeze(), #Contiene los identificadores numéricos de los tokens de la secuencia objetivo etiqueta después de ser procesados por el tokenizador
        }


In [22]:

# Crear instancia de la clase DrugRecommendationDataset
dataset_instance = DrugRecommendationDataset(dataset['train'], tokenizer, max_length=32)  # Ajusta max_length según sea necesario

# Crear el dataloader
batch_size = 16
dataloader = DataLoader(dataset_instance, batch_size=batch_size, shuffle=True) #sfuflle  garantiza que los lotes se generen en orden aleatorio en cada época de entrenamiento


In [23]:

# Definir la función fit
def fit_bart(model, dataloader, epochs=1, device="cuda"):
    model.train()
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5) #Se utiliza el optimizador AdamW para ajustar los pesos del modelo. La tasa de aprendizaje se establece en 1e-5

    for epoch in range(epochs):
        train_loss = 0.0 #Variable para acumular la pérdida durante el entrenamiento.
        bar = tqdm(dataloader) #Variable para acumular la pérdida durante el entrenamiento.

        for batch in bar:
            inputs = { #Prepara los datos de entrada para el modelo moviéndolos al dispositivo.
                "input_ids": batch["input_ids"].to(device),  #Accede a la información de los identificadores de entrada en el lote actual.
                "attention_mask": batch["attention_mask"].to(device), #Accede a la máscara de atención asociada al lote actual.
                "labels": batch["labels"].to(device), #Accede a las etiquetas asociadas al lote actual
            }

            outputs = model(**inputs) #Realiza el cálculo hacia adelante forward utilizando el modelo.
            loss = outputs.loss #Obtiene la pérdida del modelo para el lote actual.

            optimizer.zero_grad() # Limpia los gradientes acumulados en los parámetros
            loss.backward() #Calcula los gradientes retropropagando la pérdida.
            optimizer.step() #Actualiza los parámetros del modelo utilizando el optimizador.

            train_loss += loss.item() #Acumula la pérdida del lote actual.

            bar.set_postfix(epoch=epoch, loss=train_loss) #Actualiza la información mostrada en la barra de progreso con el número de época y la pérdida acumulada


In [ ]:
# Entrenar el modelo BART
fit_bart(model, dataloader)

100%|██████████| 1386/1386 [1:45:35<00:00,  4.57s/it, epoch=0, loss=1.57e+3]


In [14]:
def predict_medication_bart(model, symptoms_text, tokenizer, device="cuda"):
    model.eval()
    model.to(device)

    input_text = f"Recomendar medicamento para: {symptoms_text}"

    inputs = tokenizer(
        input_text,
        max_length=64,  # Ajusta según sea necesario
        return_tensors="pt",
        padding="max_length",
        truncation=True,
    )

    inputs = {key: val.to(device) for key, val in inputs.items()}

    outputs = model.generate(**inputs, max_length=64)  # Ajusta según sea necesario
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded_output



In [ ]:
# Ejemplo de predicción
symptoms_text = "Experiencing muscle pain, fatigue, and a persistent cough."
prediction = predict_medication_bart(model, symptoms_text, tokenizer, device="cuda")
print("Recomendación de medicamento:", prediction)


Recomendación de medicamento: Bupropion / naltrexone


In [ ]:
# guardar modelo

PATH = '/content/drive/MyDrive/chechpoint/checkpoint.pt'
torch.save(model.state_dict(), PATH)

In [7]:
def evaluate_bart(model, dataloader, device="cuda"):
    model.eval()
    model.to(device)
    total_loss = 0.0

    with torch.no_grad():
        bar = tqdm(dataloader)

        for batch in bar:
            inputs = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
                "labels": batch["labels"].to(device),
            }

            outputs = model(**inputs)
            loss = outputs.loss

            total_loss += loss.item()

            bar.set_postfix(loss=total_loss)

    return total_loss / len(dataloader)

# # Uso
# val_loss = evaluate_bart(model, val_dataloader)
# print(f"Validation Loss: {val_loss}")


In [4]:
PATH = '/content/drive/MyDrive/chechpoint/checkpoint.pt'

In [8]:
# cargar modelo

model.load_state_dict(torch.load(PATH))
model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [9]:
fit_bart(model, dataloader, epochs=1)

100%|██████████| 1386/1386 [1:46:20<00:00,  4.60s/it, epoch=0, loss=390]


In [10]:
# guardar modelo

PATH = '/content/drive/MyDrive/chechpoint/checkpoint2.pt'
torch.save(model.state_dict(), PATH)

In [16]:
# Ejemplo de predicción
symptoms_text = "Experiencing muscle pain, fatigue, and a persistent cough."
prediction = predict_medication_bart(model, symptoms_text, tokenizer, device="cuda")
print("Recomendación de medicamento:", prediction)


Recomendación de medicamento: Phentermine
